# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 18 2022 1:34PM,244502,10,MSP213249,"Methapharm, Inc.",Released
1,Aug 18 2022 1:32PM,244501,10,MSP213248,"Methapharm, Inc.",Released
2,Aug 18 2022 1:32PM,244501,10,MSP213250,"Methapharm, Inc.",Released
3,Aug 18 2022 1:32PM,244501,10,MSP213251,"Methapharm, Inc.",Released
4,Aug 18 2022 1:18PM,244498,10,8419175-SD,Eywa Pharma Inc.,Released
5,Aug 18 2022 1:12PM,244497,19,ACG-2102490993,ACG North America LLC,Executing
6,Aug 18 2022 1:12PM,244497,19,ACG-2102490994,ACG North America LLC,Released
7,Aug 18 2022 12:57PM,244496,19,ACG-2102490988,ACG North America LLC,Released
8,Aug 18 2022 12:57PM,244496,19,ACG-2102490990,ACG North America LLC,Released
9,Aug 18 2022 12:56PM,244495,10,8418580,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
55,244497,Executing,1
56,244497,Released,1
57,244498,Released,1
58,244501,Released,3
59,244502,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244496,NaN,NaN,2.0
244497,NaN,1.0,1.0
244498,NaN,NaN,1.0
244501,NaN,NaN,3.0
244502,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244396,0.0,0.0,73.0
244406,0.0,0.0,1.0
244410,0.0,0.0,1.0
244417,15.0,10.0,0.0
244418,0.0,0.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244396,0,0,73
244406,0,0,1
244410,0,0,1
244417,15,10,0
244418,0,0,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244396,0,0,73
1,244406,0,0,1
2,244410,0,0,1
3,244417,15,10,0
4,244418,0,0,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244396,,,73
1,244406,,,1
2,244410,,,1
3,244417,15,10,
4,244418,,,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 18 2022 1:34PM,244502,10,"Methapharm, Inc."
1,Aug 18 2022 1:32PM,244501,10,"Methapharm, Inc."
4,Aug 18 2022 1:18PM,244498,10,Eywa Pharma Inc.
5,Aug 18 2022 1:12PM,244497,19,ACG North America LLC
7,Aug 18 2022 12:57PM,244496,19,ACG North America LLC
9,Aug 18 2022 12:56PM,244495,10,Beach Products Inc
23,Aug 18 2022 12:52PM,244494,10,Emerginnova
24,Aug 18 2022 12:49PM,244493,10,Eywa Pharma Inc.
25,Aug 18 2022 12:44PM,244491,16,Sartorius Bioprocess Solutions
27,Aug 18 2022 12:30PM,244489,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 18 2022 1:34PM,244502,10,"Methapharm, Inc.",,,1
1,Aug 18 2022 1:32PM,244501,10,"Methapharm, Inc.",,,3
2,Aug 18 2022 1:18PM,244498,10,Eywa Pharma Inc.,,,1
3,Aug 18 2022 1:12PM,244497,19,ACG North America LLC,,1,1
4,Aug 18 2022 12:57PM,244496,19,ACG North America LLC,,,2
5,Aug 18 2022 12:56PM,244495,10,Beach Products Inc,,,14
6,Aug 18 2022 12:52PM,244494,10,Emerginnova,,,1
7,Aug 18 2022 12:49PM,244493,10,Eywa Pharma Inc.,,,1
8,Aug 18 2022 12:44PM,244491,16,Sartorius Bioprocess Solutions,,,2
9,Aug 18 2022 12:30PM,244489,12,Hush Hush,,6,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 1:34PM,244502,10,"Methapharm, Inc.",1,,
1,Aug 18 2022 1:32PM,244501,10,"Methapharm, Inc.",3,,
2,Aug 18 2022 1:18PM,244498,10,Eywa Pharma Inc.,1,,
3,Aug 18 2022 1:12PM,244497,19,ACG North America LLC,1,1,
4,Aug 18 2022 12:57PM,244496,19,ACG North America LLC,2,,
5,Aug 18 2022 12:56PM,244495,10,Beach Products Inc,14,,
6,Aug 18 2022 12:52PM,244494,10,Emerginnova,1,,
7,Aug 18 2022 12:49PM,244493,10,Eywa Pharma Inc.,1,,
8,Aug 18 2022 12:44PM,244491,16,Sartorius Bioprocess Solutions,2,,
9,Aug 18 2022 12:30PM,244489,12,Hush Hush,1,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 1:34PM,244502,10,"Methapharm, Inc.",1,,
1,Aug 18 2022 1:32PM,244501,10,"Methapharm, Inc.",3,,
2,Aug 18 2022 1:18PM,244498,10,Eywa Pharma Inc.,1,,
3,Aug 18 2022 1:12PM,244497,19,ACG North America LLC,1,1,
4,Aug 18 2022 12:57PM,244496,19,ACG North America LLC,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 1:34PM,244502,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Aug 18 2022 1:32PM,244501,10,"Methapharm, Inc.",3.0,NaN,NaN
2,Aug 18 2022 1:18PM,244498,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Aug 18 2022 1:12PM,244497,19,ACG North America LLC,1.0,1.0,NaN
4,Aug 18 2022 12:57PM,244496,19,ACG North America LLC,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 1:34PM,244502,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Aug 18 2022 1:32PM,244501,10,"Methapharm, Inc.",3.0,0.0,0.0
2,Aug 18 2022 1:18PM,244498,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Aug 18 2022 1:12PM,244497,19,ACG North America LLC,1.0,1.0,0.0
4,Aug 18 2022 12:57PM,244496,19,ACG North America LLC,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4889431,82.0,45.0,6.0
102,244450,1.0,0.0,0.0
12,733359,2.0,7.0,13.0
15,488939,14.0,25.0,1.0
16,977778,22.0,1.0,0.0
17,1222298,4.0,1.0,0.0
19,1222330,20.0,11.0,15.0
20,488855,90.0,0.0,0.0
21,1222155,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4889431,82.0,45.0,6.0
1,102,244450,1.0,0.0,0.0
2,12,733359,2.0,7.0,13.0
3,15,488939,14.0,25.0,1.0
4,16,977778,22.0,1.0,0.0
5,17,1222298,4.0,1.0,0.0
6,19,1222330,20.0,11.0,15.0
7,20,488855,90.0,0.0,0.0
8,21,1222155,4.0,1.0,0.0
9,22,488943,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,82.0,45.0,6.0
1,102,1.0,0.0,0.0
2,12,2.0,7.0,13.0
3,15,14.0,25.0,1.0
4,16,22.0,1.0,0.0
5,17,4.0,1.0,0.0
6,19,20.0,11.0,15.0
7,20,90.0,0.0,0.0
8,21,4.0,1.0,0.0
9,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,82.0
1,102,Released,1.0
2,12,Released,2.0
3,15,Released,14.0
4,16,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21,22
Status,,,,,,,,,,
Completed,6.0,0.0,13.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0
Executing,45.0,0.0,7.0,25.0,1.0,1.0,11.0,0.0,1.0,0.0
Released,82.0,1.0,2.0,14.0,22.0,4.0,20.0,90.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,6.0,0.0,13.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0
1,Executing,45.0,0.0,7.0,25.0,1.0,1.0,11.0,0.0,1.0,0.0
2,Released,82.0,1.0,2.0,14.0,22.0,4.0,20.0,90.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,6.0,0.0,13.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0
1,Executing,45.0,0.0,7.0,25.0,1.0,1.0,11.0,0.0,1.0,0.0
2,Released,82.0,1.0,2.0,14.0,22.0,4.0,20.0,90.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()